In [15]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
scraping_date = '_26_04_2021'
data = pd.read_csv('./data/full_profiles'+scraping_date+'.csv')

In [9]:
data

,name,languages,description,rating,nb_reviews,Price/hour in USD,skills,industry_expertise,nb_projects,nb_buyers,last_active,gender,city,country
0,Maria H.,['English'],We are a small UK based company speciaIising i...,4.9,4694,34.74,"['animation', 'graphics design', 'brand design...","['IT', 'Internet', 'Marketing', 'Media', 'Tech...",7672,4168,26-04-2021,female,Sheffield,United Kingdom
1,Cormac Reynolds,['English'],Looking for a link building solution or some g...,5.0,1190,138.95,"['online marketing', 'link building', 'marketi...",[],1085,533,25-04-2021,unknown,City of London,United Kingdom
2,Denise Toepel,"['English', 'Spanish', 'French']",I am a Translation Specialist. I work in publ...,0.0,0,27.79,"['English translation', 'French <=> English', ...","['Education', 'Telecommunications']",0,0,25-04-2021,female,Cuenca,Ecuador
3,Translate Guru .,[],Hello! Hola! Ciao! Bonjour! 你好!\r\n\r\nI do hi...,4.9,598,20.00,"['German <=> English translation', 'Arabic <=>...",[],721,376,26-04-2021,unknown,Cannanore,India
4,Logical Translation & Localisation,"['English', 'Spanish', 'French', 'Italian']",OFFERING A WIDE RANGE OF TRANSLATION SERVICES\...,4.9,135,27.79,"['editing', 'proofreading', 'German translatio...","['Advertising', 'Aerospace', 'Automotive', 'Co...",140,100,24-04-2021,unknown,City of London,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998,Goggle Software Ltd,['English'],Goggle Software build CRMs and bespoke portals...,5.0,31,69.47,"['ajax', 'database development', 'hosting deve...","['Electronics', 'IT', 'Technology']",26,22,15-04-2021,male,Leeds,United Kingdom
9999,Roshan Rupasinghe,['English'],I am an IIBA certified business analyst with 3...,0.0,0,12.00,"['business analysis', 'business writing', 'bus...",['IT'],0,0,02-12-2020,unknown,Colombo,Sri Lanka
10000,Sally Lavinia,"['English', 'Swahili']",NaN,0.0,0,30.00,"['microsoft excel', 'proofreading', 'academic ...","['Financial', 'Services', 'Internet', 'Legal',...",0,0,30-12-2020,female,Nairobi,Kenya
10001,Mohammed Younes,"['Arabic', 'English']","With a couple of years of experience, I was ab...",0.0,0,15.00,"['graphics design', 'social media management',...","['Advertising', 'Food', 'Media', 'Sport']",0,0,25-04-2021,male,Zgharta,Lebanon


# Number of occurrences of all spoken languages

In [20]:
available_languages = {}
languages = np.array(data.languages)
for profile_languages in languages:
    profile_languages = ast.literal_eval(profile_languages)
    if len(profile_languages)>0:
        for language in profile_languages:
            if language not in available_languages:
                available_languages[language] = 1
            else:
                available_languages[language] += 1

In [21]:
len(available_languages)

127

In [46]:
lang = np.array(list(available_languages.keys()))
count_lang = list(available_languages.values())
sorted_count_lang = np.argsort(count_lang)[-10:]
most_spoken_languages = lang[sorted_count_lang]

In [49]:
most_spoken_languages

array(['Portuguese', 'Russian', 'Urdu', 'Hindi', 'Italian', 'German',
       'Arabic', 'Spanish', 'French', 'English'], dtype='<U13')

## Create dummy variables for the 10 most spoken languages

In [56]:
n = len(data)
english, french, spanish, arabic, german, italian, hindi, urdu, russian, portuguese = np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n)

for i, lang in enumerate(languages):
    lang=ast.literal_eval(lang)
    if 'English' in lang:
        english[i]=1
    if 'French' in lang:
        french[i]=1
    if 'Spanish' in lang:
        spanish[i]=1
    if 'Arabic' in lang:
        arabic[i]=1
    if 'German' in lang:
        german[i]=1
    if 'Italian' in lang:
        italian[i]=1
    if 'Hindi' in lang:
        hindi[i]=1
    if 'Urdu' in lang:
        urdu[i]=1
    if 'Russian' in lang:
        russian[i]=1
    if 'Portuguese' in lang:
        portuguese[i]=1

# Number of occurences of all industry expertises

In [23]:
available_expertise = {}
industry_expertise = np.array(data.industry_expertise)
for profile_expertise in industry_expertise:
    profile_expertise = ast.literal_eval(profile_expertise)
    if len(profile_expertise)>0:
        for expertise in profile_expertise:
            if expertise not in available_expertise:
                available_expertise[expertise] = 1
            else:
                available_expertise[expertise] += 1

In [24]:
len(available_expertise)

66

In [48]:
exper = np.array(list(available_expertise.keys()))
count_exper = list(available_expertise.values())
sorted_count_exper = np.argsort(count_exper)[-10:]
most_imp_exper = exper[sorted_count_exper]

In [62]:
most_imp_exper

array(['Art', 'Travel', 'Entertainment', 'Technology', 'Internet', 'IT',
       'Media', 'Advertising', 'Education', 'Marketing'], dtype='<U18')

## Create dummy variables for the 10 most important industry expertise

In [65]:
marketing, education, advertising, media, it, internet, technology, entertainment, travel, art = np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n)

for i, exper in enumerate(industry_expertise):
    exper=ast.literal_eval(exper)
    if 'Marketing' in exper:
        marketing[i]=1
    if 'Education' in exper:
        education[i]=1
    if 'Advertising' in exper:
        advertising[i]=1
    if 'Media' in exper:
        media[i]=1
    if 'IT' in exper:
        it[i]=1
    if 'Internet' in exper:
        internet[i]=1
    if 'Technology' in exper:
        technology[i]=1
    if 'Entertainment' in exper:
        entertainment[i]=1
    if 'Travel' in exper:
        travel[i]=1
    if 'Art' in exper:
        art[i]=1

In [ ]:
rating = data.rating
nb_reviews = data.nb_reviews
price = data['Price/hour in USD']